In [3]:
import time
from glob import glob
import os
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import astropy
from astropy.io import fits
from astropy.table import Table
from astropy.wcs import WCS
from astropy.stats import SigmaClip
from photutils.background import Background2D, MedianBackground

from astro import astroutils
from astro import mastrometry
from astro import mimage
from astro import mphot

from utils import mutils
from utils import mlogging 

from canislib.data import pload, psave, doublesort

from astro.astroutils import jd2UTCtimestamp, UTCtimestamp2jd

log = mlogging.getLogger('photometry')

%load_ext autoreload
%autoreload 2
%matplotlib ipympl

target= ('08 19 28.99', '70 42 19.4') # SIMBAD values is correct
comps = [     
            ('08 19 35.27861', '70 42 46.68284'), # A=1774 SNR=592   stdev = 0.2 (previously #1)  
            ('08 19 42.614',   '70 40 23.9842'),    # A=4800 SNR=1078 stdev = 0.05 (previously #5)
            ('08 19 57.87424', '70 42 53.07892'),   # A=2295 SNR=775 stdev = 0.03 (previously #6)
            ('08 19 33.91390', '70 44 27.99391'),   # A=3500 SNR=904      
            ('08 19 15.57437', '70 44 42.37452'),   # A=2014 SNR=637.7
        ]

In [ ]:
"""Just aperphot (has it's own local subtraction)."""

# refer to ulx_calibration.ipynb

In [ ]:
"""aperphot with global Background2D subtraction"""

# tabled

In [7]:
"""photutils.aperture with global Background2D subtraction """
# https://photutils.readthedocs.io/en/stable/aperture.html
from photutils.aperture import CircularAperture
from photutils.aperture import aperture_photometry


for cam in ['marana', 'nd12', 'tek2k', 'acam']:
    seeings_dict = pload(f'out/seeings/seeings_{cam}.p')
    print(f'\n================== {cam} ==================')

    outdir = f'out/photometry/photutils_results_{cam}.p'
    if os.path.exists(outdir):
        continue
    files = seeings_dict['files']
    seeings = seeings_dict['seeings']
    times = seeings_dict['times']
    dmags = []

    results = []
    print(len(files))
    for i, fn in enumerate(files[0:]):
        try:
            print(f'{i} ', end="")
            objs = [target] + comps
            coords = []
            for obj in objs:
                x, y = mphot.equatorial2pix(obj, fn)
                x = x.flat[0]
                y = y.flat[0]
                coords.append((x,y))
            hdul = fits.open(fn)
            image = hdul[1].data
            sigma_clip = SigmaClip(sigma=3.0)
            bkg_estimator = MedianBackground()
            bg = Background2D(image, (50, 50), filter_size=(3, 3), 
                sigma_clip=sigma_clip, bkg_estimator=bkg_estimator)
            aperture = CircularAperture(coords, r=3.8)
            bg_removed = image-bg.background
            phot_table = aperture_photometry(bg_removed, aperture, error=np.sqrt(bg_removed))
            
            results.append(phot_table)
        except ValueError:
            print(f'{fn} FAILURE')
            results.append(np.nan)
    psave(results, outdir)


================== marana ==================

================== nd12 ==================

================== tek2k ==================

================== acam ==================
1278
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 /users/canis/dev/holmberg/data/acam_seeing_below_6.0/g17d341.187.a.fits FAILURE
147 /users/canis/dev/holmberg/data/acam_seeing_below_6.0/g17d341.193.a.fits FAILURE
148 /users/canis/dev/holmberg/data/acam_seeing_below_6.0/g17d341.194.a.fits FAILURE
149 /users/canis/dev/holmberg/data/acam_seeing_below_6.0/g17d341.197.a.fits FAILURE
150 /u

In [ ]:
hdul = fits.open(r'/Users/canis/dev/holmberg/data/marana_seeing_below_6.0/g22d147.086.a.fits')
image = hdul[1].data

In [ ]:
"""photutils.aperture with local background subtraction"""

In [ ]:
###############
# OLD STUFF
###############
saturations = {'tek2k': 28000, 'nd12': 28000, 'Marana': 56000, 'acam': 56000}
plate_scales = {'tek2k': .3252, 'nd12': .20325, 'Marana': .44, 'acam': 0.6}
dr_tek2k = 3.8
version='_v3'
def save_cntrds(cntrds, fn):
    foo = cntrds.copy()
    foo.attrs['compcoords'] = str(foo.compcoords)
    foo.to_netcdf(fn, mode='w')
print(f'{len(comps)} comps')


scope = {'tek2k' : 61, 'nd12': 61, 'Marana': 13, 'acam': 13}
for i in [2.0, 2.5, 3.0, 3.5, 4.0, 6.0]:
    for c in ['tek2k', 'nd12', 'Marana', 'acam']: # 'tek2k', 'nd12', 'Marana'
        outpath = f'/users/canis/dev/holmberg/out/{c}/seeing_below_{i}/cntrds{version}.nc'                
        if os.path.exists(outpath):
            log.info(f'{i} {c} {n} already done')
            continue

        dr = dr_tek2k * plate_scales['tek2k'] / plate_scales[c]
        bw = 7 * plate_scales['tek2k'] / plate_scales[c]
        sa = 15 * plate_scales['tek2k'] / plate_scales[c]

        log.info(f'{c} {i}')
        log.info(f'Using saturation={saturations[c]}')
        log.info(f'Using disk radius ={dr} {bw} {sa}')
        log.info(f'Using {len(comps)} comps')

        fdir = f'{HOME}/data/cameras/{c}/seeing_below_{i}'
        flist = sorted(glob.glob(f'{fdir}/*.a.nc'))
        try:
            cntrds = mphot.aperphot(flist, startcoords=target,
                                compcoords=comps, ncompmax=5, snrmin=100,
                                diskradius=dr, bufferwidth=bw, skyannulus=sa,
                                Nthresh=2.4, autoradius='fixed', emax=0.5,
                                saturation=saturations[c],
                                nhist=1, quick='noctrd', maxcpus=None, verbose=True)
            save_cntrds(cntrds, outpath)
        except Exception as e:
            print(e)
            print('HEY')


## aperphot: compstars

In [ ]:
print(len(comps))
for i in [3.0]:
    for n in range(len(comps)):
        for c in ['tek2k', 'nd12', 'Marana', 'acam']:
#             if c=='acam' and i==2.0:
#                 continue
            outpath = f'{HOME}/data/centroids/{c}/seeing_below_{i}/cntrds{version}_{n+1}.nc'

            if os.path.exists(outpath):
                log.info(f'{i} {c} {n+1} already done')
                continue

            dr = dr_tek2k * plate_scales['tek2k'] / plate_scales[c]
            bw = 7 * plate_scales['tek2k'] / plate_scales[c]
            sa = 15 * plate_scales['tek2k'] / plate_scales[c]

            log.info(f'{c} {i} {n+1}')
            log.info(f'Using saturation={saturations[c]}')
            log.info(f'Using disk radius ={dr} {bw} {sa}')

            fdir = f'{HOME}/data/cameras/{c}/seeing_below_{i}'
            flist = sorted(glob.glob(f'{fdir}/*.a.nc'))

            if n == 0:
                used_comps = comps[1:]
            elif n == len(comps) - 1:
                used_comps =  comps[:n]
            else:
                used_comps = list(np.concatenate([comps[:n], comps[n+1:]])) # avoid truth error
            try:
                cntrds = mphot.aperphot(flist, startcoords=target,
                                    compcoords=comps, ncompmax=5, snrmin=100,
                                    diskradius=dr, bufferwidth=bw, skyannulus=sa,
                                    Nthresh=2.4, autoradius='fixed', emax=0.5,
                                    saturation=saturations[c],
                                    nhist=1, quick='noctrd', maxcpus=None, verbose=True)
                save_cntrds(cntrds, outpath)
            except Exception as e:
                print(e)
                print('HEY')